In [ ]:
__author__ = 'Benjamin Weaver <benjamin.weaver@noirlab.edu>'
__version__ = '20210125'
__datasets__ = ['sdss_dr16']
__keywords__ = ['HowTo','spectra','query','visualization','SDSS']

# How to Plot Spectral Data with Prospect, Specutils and the Data Lab Spectrum Service

## Table of Contents

* [Goals](#Goals)
* [Summary](#Summary)
* [Disclaimer & Attribution](#Disclaimer-&-Attribution)
* [Imports & Setup](#Imports-&-Setup)
* [References & Resources](#References-&-Resources)

## Goals

Obtain SDSS spectra using the Data Lab spectrum service, convert data to [specutils](https://github.com/astropy/specutils) objects as needed, and use [prospect](https://github.com/desihub/prospect) to display the data.

## Summary

Prospect is an *interactive* spectrum visualization service that is being actively used by the DESI project for visual inspection of commissioning and survey validation spectra right now.  Rather than just a simple *flux* versus *wavelength* plot, the interactive spectrum display provides pan, zoom, known spectral lines, targeting and other catalog information, metadata, *etc.*  The [Bokeh](https://bokeh.org) library handles the low-level interactivity.

We can leverage the generic spectrum container objects provided by [specutils](https://github.com/astropy/specutils) to allow Prospect to plot data from other surveys.  In this example, we are using [SDSS DR16](https://www.sdss.org/dr16/).

In the future we will extend this functionality to additional instruments.  For example, we are excited by the possibility of using Prospect and the Data Lab spectrum service for Gemini spectra.

## Disclaimer & Attribution

If you use this notebook for your published science, please acknowledge the following:

* Data Lab concept paper: Fitzpatrick et al., "The NOAO Data Laboratory: a conceptual overview", SPIE, 9149, 2014, http://dx.doi.org/10.1117/12.2057445
* Data Lab disclaimer: https://datalab.noirlab.edu/disclaimers.php

## Imports & Setup

**TODO**: final clean-up needed when prospect is installed on the production server.

In [1]:
import os
import sys
# Manually install prospect for now.
sys.path.insert(0, os.path.join(os.environ['HOME'], 'Documents', 'Code', 'git', 'desihub', 'prospect', 'py'))
import numpy as np
import astropy.units as u
from astropy.table import Table
from astropy.nddata import InverseVariance
from specutils import Spectrum1D, SpectrumCollection, SpectrumList
from prospect.specutils import read_spPlate, read_spZbest
from prospect.plotspecutils import plotspectra
from dl import queryClient as qc, storeClient as sc, specClient as spec

## SDSS/eBOSS spPlate file

SDSS spectra are stored per-plate in spPlate files.  These contain 640 spectra for the original SDSS spectrograph or 1000 spectra for the BOSS/eBOSS spectrograph.  All spPlate files have a common wavelength solution, so a spPlate file can be represented by a [Spectrum1D](https://specutils.readthedocs.io/en/stable/api/specutils.Spectrum1D.html#specutils.Spectrum1D) object.  This object is an input to the visualization system, so we need to get outputs from the spectrum service that match the inputs to the visualization.

## Find Plate 2955 in the Spectrum Service

### Plate Location

Here we find spectra corresponding to SDSS plate 2955, but using the spectrum service instead of reading the spectra from a file.

In [4]:
plate = 2955
# What the heck? Can't use a semicolon any longer?
# q = "SELECT ra, dec FROM sdss_dr16.platex WHERE plate = %d;" % plate
q = "SELECT ra, dec FROM sdss_dr16.platex WHERE plate = {0:d}".format(plate)
r = qc.query(sql=q, fmt='array')
r

array([235.4204  ,   1.644156])

### Spectrum Identifiers

SDSS plates are circular, so we'll do a cone search around the nominal plate center.  This gets us the spectrum identifiers we need to retrieve the spectra themselves.

In [3]:
sdss_ids = spec.query(r[0], r[1], 1.5, fmt='array', out='', constraint='plate=2955 ORDER BY specobjid LIMIT 50')
sdss_ids

array([3327035125891360768, 3327036500280895488, 3327037050036709376,
       3327038424426244096, 3327038974182057984, 3327040073693685760,
       3327040348571592704, 3327040623449499648, 3327040898327406592,
       3327041173205313536, 3327041722961127424, 3327041997839034368,
       3327042822472755200, 3327043097350662144, 3327043372228569088,
       3327043921984382976, 3327044196862289920, 3327045021496010752,
       3327045296373917696, 3327046121007638528, 3327046670763452416,
       3327047495397173248, 3327048045152987136, 3327048320030894080,
       3327048594908801024, 3327048869786707968, 3327049144664614912,
       3327049419542521856, 3327049694420428800, 3327050244176242688,
       3327051068809963520, 3327051343687870464, 3327052443199498240,
       3327052992955312128, 3327053267833219072, 3327053817589032960,
       3327054092466939904, 3327054642222753792, 3327054917100660736,
       3327055191978567680, 3327055466856474624, 3327056016612288512,
       3327056566368

### Spectrum Service: Flux versus Wavelength

Now we fetch the spectra.  The returned data will be compacted into a single `Spectrum1D` object.

In [5]:
%%time
sdss = spec.getSpec(sdss_ids, fmt='Spectrum1D', align=True)
sdss

CPU times: user 99.5 ms, sys: 54.3 ms, total: 154 ms
Wall time: 603 ms


<Spectrum1D(flux=<Quantity [[0., 0., 0., ..., 0., 0., 0.],
           [0., 0., 0., ..., 0., 0., 0.],
           [0., 0., 0., ..., 0., 0., 0.],
           ...,
           [0., 0., 0., ..., 0., 0., 0.],
           [0., 0., 0., ..., 0., 0., 0.],
           [0., 0., 0., ..., 0., 0., 0.]] erg / (Angstrom cm2 s)>, spectral_axis=<SpectralCoord [3802.77  , 3803.6448, 3804.522 , ..., 9217.223 , 9219.344 ,
                9221.464 ] Angstrom, 
	radial_velocity=0.0 km / s, 
	redshift=0.0, 
	doppler_rest=0.0 Angstrom, 
	doppler_convention=None, 
	observer=None, 
	target=None>, uncertainty=InverseVariance([[0., 0., 0., ..., 0., 0., 0.],
                 [0., 0., 0., ..., 0., 0., 0.],
                 [0., 0., 0., ..., 0., 0., 0.],
                 ...,
                 [0., 0., 0., ..., 0., 0., 0.],
                 [0., 0., 0., ..., 0., 0., 0.],
                 [0., 0., 0., ..., 0., 0., 0.]]))>

### Redshift and other Metadata

We also need target and redshift information, which is stored in the SpecObj catalog.  Although all information is extractable from the database, we need to reformat it to match the expectations of Prospect.

In [6]:
q = "SELECT specobjid, class, subclass, z, zerr, rchi2diff, zwarning FROM sdss_dr16.specobj WHERE specobjid IN ({0}) ORDER BY specobjid".format(','.join(map(str, sdss_ids.tolist())))
sdss_z = qc.query(sql=q, fmt='table')
for c in ('specobjid', 'class', 'subclass', 'z', 'zerr', 'rchi2diff', 'zwarning'):
    if c == 'zerr':
        sdss_z.rename_column(c, 'Z_ERR')
    else:
        sdss_z.rename_column(c, c.upper())
sdss_z

SPECOBJID,CLASS,SUBCLASS,Z,Z_ERR,RCHI2DIFF,ZWARNING
int64,str6,str11,float64,float64,float64,int64
3327035125891360768,GALAXY,--,0.262199,6.42647e-05,0.161533,0
3327036500280895488,GALAXY,--,0.21097,5.79199e-05,0.139213,0
3327037050036709376,STAR,K5,-2.34944e-06,1.43502e-05,0.496992,0
3327038424426244096,GALAXY,--,0.260543,6.22988e-05,0.225588,0
3327038974182057984,GALAXY,--,0.112768,1.59947e-05,0.510905,0
3327040073693685760,GALAXY,--,0.0636493,2.83125e-05,0.424366,0
3327040348571592704,GALAXY,--,0.208361,4.18928e-05,1.17941,0
3327040623449499648,GALAXY,--,0.0,0.0,0.0,134
3327040898327406592,GALAXY,--,0.207845,4.12366e-05,0.619729,0


In [7]:
q = """SELECT s.specobjid, s.targetobjid, s.ra, s.dec,
    s.primtarget, s.sectarget,
    s.boss_target1,
    s.ancillary_target1, s.ancillary_target2,
    s.eboss_target0, s.eboss_target1, s.eboss_target2,
    p.u, p.g, p.r, p.i, p.z
FROM sdss_dr16.specobj AS s
LEFT JOIN sdss_dr16.photoplate AS p
ON s.bestobjid = p.objid
WHERE specobjid IN ({0})
ORDER BY specobjid""".format(','.join(map(str, sdss_ids.tolist())))
sdss_plugmap = qc.query(sql=q, fmt='table')
for c in ('ra', 'dec', 'primtarget', 'sectarget',
          'boss_target1',
          'ancillary_target1', 'ancillary_target2',
          'eboss_target0', 'eboss_target1', 'eboss_target2'):
    sdss_plugmap.rename_column(c, c.upper())
sdss_plugmap['OBJID'] = np.vstack((np.bitwise_and(sdss_plugmap['targetobjid'] >> 32, 2**16 - 1).data,
                                   np.bitwise_and(sdss_plugmap['targetobjid'] >> 48, 2**11 - 1).data,
                                   np.bitwise_and(sdss_plugmap['targetobjid'] >> 29, 2**3 - 1).data,
                                   np.bitwise_and(sdss_plugmap['targetobjid'] >> 16, 2**12 - 1).data,
                                   np.bitwise_and(sdss_plugmap['targetobjid'], 2**16 - 1).data)).T
sdss_plugmap['MAG'] = np.vstack((np.where(np.isnan(sdss_plugmap['u'].data), 0, sdss_plugmap['u'].data),
                                 np.where(np.isnan(sdss_plugmap['g'].data), 0, sdss_plugmap['g'].data),
                                 np.where(np.isnan(sdss_plugmap['r'].data), 0, sdss_plugmap['r'].data),
                                 np.where(np.isnan(sdss_plugmap['i'].data), 0, sdss_plugmap['i'].data),
                                 np.where(np.isnan(sdss_plugmap['z'].data), 0, sdss_plugmap['z'].data))).T

sdss.meta['plugmap'] = sdss_plugmap
sdss_plugmap

specobjid,targetobjid,RA,DEC,PRIMTARGET,SECTARGET,BOSS_TARGET1,ANCILLARY_TARGET1,ANCILLARY_TARGET2,EBOSS_TARGET0,EBOSS_TARGET1,EBOSS_TARGET2,u,g,r,i,z,OBJID [5],MAG [5]
int64,int64,float64,float64,int64,int64,int64,int64,int64,int64,int64,int64,float64,float64,float64,float64,float64,int64,float64
3327035125891360768,11268994537292320,236.58816,0.91476191,96,0,0,0,0,0,0,0,21.3034,19.1014,17.512,16.9372,16.5634,2327 .. 544,21.3034 .. 16.5634
3327036500280895488,11268994537161253,236.31498,0.83142165,64,0,0,0,0,0,0,0,20.5745,19.0677,17.9301,17.3995,16.9857,2327 .. 549,20.5745 .. 16.9857
3327037050036709376,11268994537095484,236.17657,0.91950364,64,0,0,0,0,0,0,0,19.8611,18.5047,17.7639,17.4456,17.246,2327 .. 316,19.8611 .. 17.246
3327038424426244096,11268994537292314,236.57992,0.84114607,32,0,0,0,0,0,0,0,21.9935,19.606,18.0296,17.4561,17.0533,2327 .. 538,21.9935 .. 17.0533
3327038974182057984,11268994537292219,236.63067,0.85355411,64,0,0,0,0,0,0,0,19.6481,18.0843,17.2542,16.8369,16.5255,2327 .. 443,19.6481 .. 16.5255
3327040073693685760,11265262790246829,236.57247,1.3802039,64,0,0,0,0,0,0,0,20.0069,18.0379,17.1379,16.7259,16.3942,1458 .. 429,20.0069 .. 16.3942
3327040348571592704,11265262790312240,236.66305,1.4061234,96,0,0,0,0,0,0,0,20.1158,17.9848,16.4976,15.9314,15.5032,1458 .. 304,20.1158 .. 15.5032
3327040623449499648,11265262252261744,234.00554,1.1758617,64,0,0,0,0,0,0,0,--,--,--,--,--,1458 .. 368,0.0 .. 0.0
3327040898327406592,11268995074162967,236.57875,1.3024246,64,0,0,0,0,0,0,0,21.2316,18.8937,17.5176,16.9796,16.5934,2327 .. 279,21.2316 .. 16.5934


## Launch the Visualization

In [ ]:
plotspectra(sdss.new_flux_unit(u.Unit('1e-17 erg/(cm**2 s Angstrom)')), zcatalog=sdss_z, model=(sdss.spectral_axis.value, sdss.meta['model']),
            notebook=True, title=os.path.basename('SDSS Plate %d' % plate),
            model_from_zcat=False, with_coaddcam=False, mask_type='PRIMTARGET', with_thumb_tab=False, with_vi_widgets=False)

## SDSS Spectra and Redshifts from Files

For reference, and to compare timing, below is how one would fetch spectra from a file.  In addition, we need the redshift catalog.

In [ ]:
os.environ['SPECTRO_REDUX'] = 'sdss_dr16://' + os.path.join('sdss', 'spectro', 'redux')
run2d = '26'
plate = '2955'
mjd = '54562'
sdss_spectra = os.path.join(os.environ['SPECTRO_REDUX'], run2d, plate, f'spPlate-{plate}-{mjd}.fits')
sdss_redshifts = os.path.join(os.environ['SPECTRO_REDUX'], run2d, plate, f'spZbest-{plate}-{mjd}.fits')
print(sdss_spectra)
print(sdss_redshifts)

In [ ]:
%%time
sdss_file = read_spPlate(sc.get(sdss_spectra, mode='fileobj'), limit=50)
sdss_file

In [ ]:
sdss_z, sdss_model = read_spZbest(sc.get(sdss_redshifts, mode='fileobj'), limit=50)

# References & Resources

* [Getting Started with Spectral Data](https://github.com/noaodatalab/specserver/blob/master/doc/03_GettingStartedWithSpectra.ipynb)